In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
import regex as re
import unicodedata
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx 
from collections import Counter
from fa2 import ForceAtlas2
from Functions import DataCleaning

In [2]:
params = {
        "thriller": {"years": ["2020s","2010s", "2000s"],
                "sub_years": [["2020", "2021", "2022"],["2010", "2011", "2012","2013","2014","2015","2016","2017","2018","2019"],[[]]]}, 
        "action": {"years": ["2020s","2010s", "2000s"],
                "sub_years": [["2020", "2021", "2022"],["2010", "2011", "2012","2013","2014","2015","2016","2017","2018","2019"],["2000", "2001", "2002","2003","2004","2005","2006","2007","2008","2009"]]},             
        "comedy": {"years": ["2020s","2010s", "2000s"],
                "sub_years": [["2020", "2021", "2022"],["2010", "2011", "2012","2013","2014","2015","2016","2017","2018","2019"],["2000", "2001", "2002","2003","2004","2005","2006","2007"]]},
        "adventure": {"years": ["2020s","2010s", "2000s"],
                "sub_years": [["2020", "2021", "2022", "2023"], ["2010", "2011", "2012","2013","2014","2015","2016","2017","2018","2019"], ["2000", "2001", "2002","2003","2004","2005","2006","2007","2008","2009"]]}, 
        "horror": {"years": ["2020", "2021", "2022", "2010","2011","2012","2013","2014","2015","2016","2017","2018","2019", "2000", "2001", "2002","2003","2004","2005","2006","2007","2008","2009"], 
                "sub_years": [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]},
        "fantasy": {"years": ["2020s","2010s", "2000s"],
                "sub_years": [[[]],[[]],[[]]]}, 
        "science_fiction": {"years": ["2020s", "2010s", "2000s"],
                "sub_years": [[[]],[[]],[[]]]}, 
        }

data_collection = DataCollection()
data_movie_artist = pd.DataFrame()
for genre in params.keys():
    for years, sub_years in zip(params[genre]["years"],params[genre]["sub_years"]):
        data = data_collection.collect_movies_artist_data(genre, years, sub_years)
        data_movie_artist = pd.concat([data_movie_artist, data]) 

data_movie_artist = data_movie_artist.reset_index(drop=True)

NameError: name 'DataCollection' is not defined

In [2]:
data_movie_artist = pd.read_json('data_movie_artist.json', orient='table')
data_movie_artist.shape

(7456, 7)

In [3]:
data_cleaner = DataCleaning(data_movie_artist)
data_cleaner.data_cleaning()
data_movie_artist_cleaned = data_cleaner.data


# Data Cleaning

In [4]:
data_movie_artist_cleaned.shape

(5780, 7)

In [5]:
data_movie_artist_cleaned['Country'].value_counts()

united states                   3372
Mix                              581
japan                            211
united kingdom                   192
europe                           149
france                           146
south korea                      114
china                            110
asia                             101
india                            100
canada                            99
hong kong                         89
philippines                       86
united kingdom united states      84
china hong kong                   62
australia                         59
spain                             58
germany                           53
canada united states              43
russia                            40
south america                     21
north nmerica                     10
Name: Country, dtype: int64

In [6]:
data_movie_artist_cleaned[data_movie_artist_cleaned.duplicated(subset=['Title'])]

,Title,Director,Cast,Country,Genre,Year,Hyperref


In [7]:
data_movie_artist_cleaned[data_movie_artist_cleaned['Title']=='The Rhythm Section']

,Title,Director,Cast,Country,Genre,Year,Hyperref
5907,The Rhythm Section,reed morano,"blake lively,jude law,sterling k brown",united states,Thriller,2020,The_Rhythm_Section


In [8]:
data_movie_artist_cleaned['Title'].to_csv('titles.csv')

In [ ]:
def clean_text_title_column(self, text):
    # Remove text within square brackets and parentheses
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\(.*?\)', '', text)

    # Replace 
    text = text.replace('!', '')
    text = text.replace('.', '')
    return text



def clean_text_cast_column(self, text):
    # Remove text within square brackets and parentheses
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\(.*?\)', '', text)

    # Replace 
    text = text.replace('|', '')
    text = text.replace('’', "'")
    text = text.replace('.', '')

    # Remove text within double and single quotes
    text = re.sub(r"\".*?\"", '', text)
    text = re.sub(r"'.*?'", '', text)    
    return text

def clean_text(self, column, lower=True):
    self.data[column] = self.data[column].str.replace(r'([a-z])([A-Z])', self.add_whitespace, regex=True) # adds space between concatenated words
    self.data[column] = self.data[column].apply(self.change_special_letters) # replace speciel letters with normal letters
    self.data[column] = self.data[column].str.strip()
    self.data[column] = self.data[column].str.replace(r'\s+', ' ', regex=True) # removes duplicated whitespaces
    if lower: 
        self.data[column] = self.data[column].str.lower() # lowercase for all besides 


def clean_conditions(self):
    conditions = (
        (self.data['Cast'] != '') &
        (self.data['Title'] != '') &
        (self.data['Cast'] != 'Canada') & 
        (self.data['Title'] != 'citation needed') &
        (self.data['Title'] != 'Kevin VanHook') &
        (self.data['Title'] != 'J.T. Petty') &
        (self.data['Title'] != 'Reggie Bannister michael hoffman jr')
    )
    self.data = self.data[conditions].reset_index(drop=True)

    # These titles with the corresponding director is wrong entries. 
    incorrect_entries = [
        ('Hot Tub Time Machine', 'sean anders john morris'),
        ('The Matrix Reloaded', 'the wachowskis nb 9'),
        ('The Matrix Revolutions', 'the wachowskis nb 10'),
        ('Jade Warrior', 'tommi eronen'),
        ('The Bleeding', 'charles picerni'),
        ('The Huntsman: Winter\'s War', 'frank darabont'),
        ('Stowaway', 'adam lipsius'), 
        ('World War Z', 'chris la martina')]
    
    for title, wrong_director in incorrect_entries:
        idx_to_remove = self.data[(self.data['Title'] == title) & 
                                    (self.data['Director'].str.lower() == wrong_director.lower())].index
        self.data = self.data.drop(idx_to_remove)

    # Define movies to remove due to wrong year
    movies_to_remove_year = {
                    'Run Sweetheart Run': "2022",
                    'The Black Phone': "2022",
                    'Bhool Bhulaiyaa 2': "2021",
                    'Apartment 143': "2012",
                    'Underworld: Blood Wars': "2017", 
                    'Bloody Bloody Bible Camp': '2012', 
                    'Flash Point': '2006', 
                    'Kingsman: The Secret Service': '2015', 
                    'Battle Royale': '2001',
                    'Smokin\' Aces': '2007',
                    'Tokyo Gore Police': '2007',
                    'Sky Captain and the World of Tomorrow': '2003',
                    'Decoys': '2003', 
                    'How to Talk to Girls at Parties': '2018',
                    'Monsters: Dark Continent': '2014', 
                    'Kingsman: The Secret Service': '2015', 
                    'Donkey Punch': '2007', 
                    'Manborg': '2010', 
                    'Army of Frankensteins': '2014',
                    'Growth':'2009', 
                    'Universal Soldier: Regeneration': '2010',
                    'Army of Frankensteins': '2014',
                    'BloodRayne': '2006',
                    '300': '2007', 
                    'M3GAN': '2023', 
                    'Excision': '2008', 
                    'Color Out of Space': '2020',
                    'Faust: Love of the Damned': '2001',
                    'Bunshinsaba': '2012',
                    'An American Haunting': '2006',
                    'The Gingerdead Man': '2006', 
                    'Big Bad Wolf': '2007',
                    'Hurt': '2008',
                    'Strigoi': '2009',
                    'Seventh Son': '2015', 
                    'The Shape of Water': '2018', 
                    'Beowulf & Grendel': '2006', 
                    'Dirty Deeds': '2002', 
                    'Beowulf Grendel': '2006'}
    
    movies_to_remove_hyperref = {'The Beach': 'The_Beach_(2000_film)', 
                                    'Edge of Tomorrow': 'Edge_of_Tomorrow_(film)', 
                                    'The Medallion': 'The_Medallion_(film)'}
        
    # Create a mask for all movies to be removed at once
    mask_year = self.data.apply(lambda x: (x['Title'], x['Year']) in movies_to_remove_year.items(), axis=1)
    self.data = self.data[~mask_year].reset_index(drop=True)

    mask_hyperref = self.data.apply(lambda x: (x['Title'], x['Hyperref']) in movies_to_remove_hyperref.items(), axis=1)
    self.data = self.data[~mask_hyperref].reset_index(drop=True)

In [ ]:
def clean_text_director_column(self, text):
    # Remove text within square brackets and parentheses
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\(.*?\)', '', text)

    # Replace + and & with a comma
    text = text.replace('+', ',')
    text = text.replace('&', ',')
    text = text.replace('-', ' ')
    text = text.replace('—', ' ')
    text = text.replace('.', '')

    # Remove text within double and single quotes
    text = re.sub(r"\".*?\"", '', text)
    text = re.sub(r"'.*?'", '', text)    
    return text

def consolidate_directors(self):
    def inner_consolidate_directors(group):
            # Find the longest director name
        longest_director = max(group, key=len)
        # Split the longest name into a set of words for easy comparison
        longest_director_words = set(longest_director.lower().split())

        # Function to check if any part of a shorter name is in the longest name
        def is_subname_any(shorter, longer_words):
            shorter_words = set(shorter.lower().split())
            # Check if any word from the shorter name is in the longer name
            return any(word in longer_words for word in shorter_words)

        # Consolidate director names
        group = [longest_director if is_subname_any(d, longest_director_words) else d for d in group]
        return group
    self.data['Director'] = self.data.groupby('Title')['Director'].transform(inner_consolidate_directors)

def data_clean_director(self):
    self.data.apply(clean_text_director_column)


In [ ]:
def differentiate_same_title(self):
    different_directors = self.data[self.data.duplicated(subset=['Title'], keep=False) &
                                    self.data.groupby(['Title'])['Director'].transform('nunique').ne(1)]
    for index, row in different_directors.iterrows():
        self.data.loc[index, 'Title'] = row['Title'] + ' ' + row['Director']
    self.data = self.data.reset_index(drop=True)

In [ ]:
def data_clean_genres(self):
    # Define your specific genre combinations and their new names
    genre_combinations = {
                        ('horror',): 'Horror',
                        ('comedy',): 'Comedy',
                        ('action',): 'Action',
                        ('thriller',): 'Thriller',
                        ('science_fiction',): 'Science Fiction',
                        ('adventure',): 'Adventure',
                        ('fantasy',): 'Fantasy',
                        ('action', 'science_fiction'): 'Science Fiction',
                        ('action', 'thriller'): 'Action',
                        ('action', 'comedy'): 'Action',
                        ('adventure', 'fantasy'): 'Fantasy',
                        ('comedy', 'horror'): 'Horror',
                        ('comedy', 'fantasy'): 'Fantasy',
                        ('horror', 'science_fiction'): 'Science Fiction',
                        ('horror', 'thriller'): 'Horror',
                        ('adventure', 'science_fiction'): 'Science Fiction',
                        ('action', 'adventure'): 'Action',
                        ('action', 'adventure', 'science_fiction'): 'Science Fiction',
                        ('adventure', 'comedy'): 'Comedy'}

    # Group by title and apply a set to the 'Genre' column
    title_genres = self.data.groupby('Title')['Genre'].apply(set).reset_index()

    # Convert the set of genres to a sorted tuple
    title_genres['genre_combination'] = title_genres['Genre'].apply(lambda x: tuple(sorted(x)))

    # Map the genre combinations to the desired string or 'other'
    title_genres['Genre'] = title_genres['genre_combination'].apply(
        lambda genre_tuple: genre_combinations.get(genre_tuple, 'Mix'))

    # Map titles to their new genre
    title_to_new_genre = pd.Series(title_genres['Genre'].values, index=title_genres['Title']).to_dict()
    self.data['Genre'] = self.data['Title'].map(title_to_new_genre)

In [ ]:
def clean_text_country_column(self, text):
    # Remove text within square brackets and parentheses
    text = re.sub(r'\[.*?\]', '', text)

    # Replace 
    text = text.replace('-', ',')
    text = text.replace('<', "")
    text = text.replace('.', '')
    text = text.replace('\{', '')
    return text

def unique_country_combinations(data):
    unique_combinations = {}
    unique_country_combinations = []
    for index, row in data.iterrows():
        country_combination = ' '.join(sorted(row['Country'].split()))
        if country_combination not in unique_combinations:
            unique_combinations[country_combination] = row['Country']
            unique_country_combinations.append(country_combination)
        else:
            first_occurrence_index = unique_country_combinations.index(country_combination)
            unique_country_combinations.append(unique_country_combinations[first_occurrence_index])
    data['Country'] = [unique_combinations[combination] for combination in unique_country_combinations]
    return data

def map_country_to_continent(self):
    country_counts = self.data['Country'].value_counts()
    continent_mapping = {
        'North America': ['united states', 'canada', 'mexico', 'honduras'],
        'United States': ['usa', 'american'],
        'Asia': ['indonesia', 'japan', 'taiwan', 'hong kong', 'china', 'south korea', 'myanmar', 'india', 'israel', 'jordan', 'qatar', 'japan', 'thailand', 'singapore', 
                'malaysia', 'kazakhstan', 'vietnam', 'pakistan', 'philippines', 'bangladesh', 'bhutan', 'cambodia', 'laos', 'brunei', 'timor-leste', 'mongolia', 
                'tajikistan', 'kyrgyzstan', 'turkmenistan', 'uzbekistan', 'kazakhstan', 'nepal'],
        'Europe': ['italy', 'germany', 'france', 'norway', 'sweden', 'ireland', 'new zealand', 'netherlands', 'denmark', 'united kingdom', 'spain', 'belgium', 'poland', 
                'czech republic', 'russia', 'austria', 'switzerland', 'iceland', 'greece', 'romania', 'serbia', 'turkey', 'luxembourg', 'portugal', 'malta', 
                'bulgaria', 'ireland', 'croatia', 'slovenia', 'slovakia', 'latvia', 'estonia', 'hungary', 'belarus', 'lithuania', 'macedonia', 'monaco', 
                    'armenia', 'kazakhstan', 'poland', 'estonia', 'hungary', 'lithuania', 'slovenia', 'slovakia', 'estonia'],
        'South America': ['argentina', 'chile', 'peru', 'brazil', 'colombia', 'ecuador', 'french guiana', 'trinidad and tobago', 'venezuela', 'guyana', 'suriname']}

    def inner_map_country_to_continent(country):
        if country_counts.get(country, 0) < 45:
            for continent, countries in continent_mapping.items():
                if country in countries:
                    return continent
            return "Mix"
        return country
    self.data['Country'] = self.data['Country'].apply(inner_map_country_to_continent)
    

def clean_country(self): 
    # Identifying rows with the longest 'Country' for each 'Title'
    different_country = self.data[self.data.duplicated(subset=['Title'], keep=False) & 
                                    self.data.groupby(['Title'])['Country'].transform('nunique').ne(1)]
    
    # Remove rows with 'other' country, except those identified above
    only_other_country = self.data.groupby('Title').filter(lambda x: (x['Country'] == 'other').all())
    different_country = different_country[~((different_country['Country'] == 'other') & 
                                            ~different_country['Title'].isin(only_other_country['Title']))]
    
    # Identify the row with the longest country name for each title
    max_length_indices = different_country.groupby('Title')['Country'].apply(lambda x: x.str.len().idxmax())
    rows_to_keep = different_country.loc[max_length_indices]

    # First, remove all rows with the titles that have duplicates in different_country
    self.data = self.data[~self.data['Title'].isin(different_country['Title'])]
    
    # Then, append the rows to keep to the filtered DataFrame
    self.data = self.data.append(rows_to_keep).reset_index(drop=True)

def data_clean_country(self):
    self.data = self.data.apply(self.clean_text_country_column)
    self.clean_text('Country')
    self.unique_country_combinations()
    self.map_country_to_continent()
    self.clean_country()

In [177]:
def drop_duplicates(self): 
    self.data = self.data.drop_duplicates(keep='first')